# Desafio 1: Properti

## Objetivos:
-  Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.

-  Realizar un análisis descriptivo de las principales variables.

- Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.

## Preparacion de ambiente
### Importar Librerias
**Numpy** y **Pandas** son importadas para administrar todas las operaciones analiticas 

**IPython** es utilizados para administrar la visualizacion de los datos

In [179]:
import numpy as np
import pandas as pd
from IPython.display import display

### Importar archivo CSV
Utilizando Pandas importamos el archivo CSV considerando la comma como delimitador 

In [180]:
dfprop=pd.read_csv(filepath_or_buffer='properatti.csv',delimiter=',',index_col = 0 )

## Visualizacion
### Visualizar la forma del Dataframe
Se utilizara la funcion nativa **shape** de la clase DataFrame para visualizar:
- Cantidad de observaciones 
- Cantidad de variables

In [181]:
dfprop.shape

(121220, 25)

### Visualizar estructura del Dataframe
Se utilizara la funcion nativa **info()** de la clase DataFrame para visualizar:
- Cantidad de variables
- Nombre de variables
- Cantidad de registros con valores **No nulos**
- Tipo de dato

In [182]:
dfprop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121220 entries, 0 to 121219
Data columns (total 25 columns):
operation                     121220 non-null object
property_type                 121220 non-null object
place_name                    121197 non-null object
place_with_parent_names       121220 non-null object
country_name                  121220 non-null object
state_name                    121220 non-null object
geonames_id                   102503 non-null float64
lat-lon                       69670 non-null object
lat                           69670 non-null float64
lon                           69670 non-null float64
price                         100810 non-null float64
currency                      100809 non-null object
price_aprox_local_currency    100810 non-null float64
price_aprox_usd               100810 non-null float64
surface_total_in_m2           81892 non-null float64
surface_covered_in_m2         101313 non-null float64
price_usd_per_m2              68617 n

## Visualizacion de valores Nan en cada variable

Visualizamos la cantidad de valores Nan para cada variable  

In [183]:
dfprop[dfprop.isnull().any(axis=1)].isna().sum()

operation                       0
property_type                   0
place_name                     23
place_with_parent_names         0
country_name                    0
                            ...  
expenses                   106958
properati_url                   0
description                     2
title                           0
image_thumbnail              3112
Length: 25, dtype: int64

## Limpieza de datos
### Eliminacion de inmuebles sin valor analitico

Eliminamos los inmuebles sin precio ni ubicaciones, ya que no tenemos ninguna forma de estimar un precio.

Condicion:
- price == Nan
- price_aprox_local_currency == Nan
- price_aprox_usd == Nan
- surface_total_in_m2 == Nan
- surface_covered_in_m2 == Nan


Los registros eliminados son 4733 

In [184]:
# Generamos una mascara para encontrar todos los registros bajo esta condicion
dfprop_withoutprice_withoutsurface=((dfprop['price'].isna()==True)\
                                    &(dfprop['price_aprox_local_currency'].isna()==True)\
                                    &(dfprop['price_aprox_usd'].isna()==True)\
                                    &(dfprop['surface_total_in_m2'].isna()==True)\
                                    &(dfprop['surface_covered_in_m2'].isna()==True))

# Instanciamos el Dataframe excluyendo los registros que no cumple con dicha condicion
dfprop=dfprop.loc[~dfprop_withoutprice_withoutsurface]


Visualizamos la forma para confirmar que se hayan eliminado los registros

In [185]:
dfprop.shape

(116487, 25)

### Separar en un nuevo Dataframe los inmuebles cuya concentracion por provincia sea menor a 30 operaciones 

Dejaremos en un nuevo DataFrame denominado 'dfprop_fewlocations' las propiedades cuya agrupacion por provincia sea menor a 30 por no tener un valor referencial para la estimacion de precios

In [186]:
#Creamos dataframe con un valor booleano que indica si la provincia tiene menos de 30 propiedades (True) o no (False).
dfprop_stateless30locations=dfprop.groupby('state_name')[['operation']].count()<30

# Renombramos la columna  
dfprop_stateless30locations.rename(columns={'state_name':'state_name','operation':'less_30_locations'},inplace=True)

# Hacemos un merge con dfprop para marcar cada una de las propiedades
dfprop=pd.merge(dfprop,dfprop_stateless30locations,how='left',on='state_name')
    
# Generamos un nuevo Dataframe para almacenar las propiedades correspondientes a provincias con menos de 30 propiedades
dfprop_fewlocations=dfprop.loc[dfprop['less_30_locations']==True]

# Eliminamos del dataframe proncipal dichas ubiaciones
dfprop=dfprop.loc[dfprop['less_30_locations']==False]

#Eliminamos las columnas de ambos dataframes
dfprop_fewlocations.drop(['less_30_locations'],axis=1,inplace=True)
dfprop.drop(['less_30_locations'],axis=1,inplace=True)

#Verificamos la cantidad de ubicaciones dentro del dataframe auxiliar
dfprop_fewlocations.shape

(74, 25)

In [187]:
#Corroboramos que se hayan eliminado las propiedades del dataframe principal
dfprop.shape

(116413, 25)

Eliminamos propiedad en moneda distinta a ARS o USD (se excluye 'NaN' siendo que tendran tratamiento mas adelante) 

In [188]:
dfprop['currency'].unique()


array(['USD', nan, 'ARS', 'PEN', 'UYU'], dtype=object)

In [189]:
dfprop=dfprop.loc[((dfprop['currency']!='UYU')&(dfprop['currency']!='PEN'))]

#Corroboramos que se hayan eliminado las propiedades del dataframe principal
dfprop['currency'].unique()

array(['USD', nan, 'ARS'], dtype=object)

Eliminamos la categoria 'store', siendo que nuestro analisis será para viviendas de ocupacion permanente.

In [190]:
dfprop['property_type'].unique()

array(['PH', 'apartment', 'house', 'store'], dtype=object)

In [191]:
dfprop=dfprop.loc[(dfprop['property_type']!='store')]

In [192]:
dfprop['property_type'].unique()

array(['PH', 'apartment', 'house'], dtype=object)

# Imputacion
## Variable place_name - valores Nan:  
- Analizamos los valores NaN de la columna 'place_name':

In [193]:
dfprop.loc[(dfprop['place_name'].isna() == True),['place_name','place_with_parent_names']]

,place_name,place_with_parent_names
6201,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
9686,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
10855,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
14066,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
17672,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
...,...,...
55036,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
55996,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
59189,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||
59191,NaN,|Argentina|Bs.As. G.B.A. Zona Norte|Tigre||


- Vemos que, de acuerdo a 'place_name_with_parents', todos los valores faltantes son 'Tigre'.
- Por lo tanto, procedemos a reemplazarlos:

In [194]:
dfprop['place_name'].fillna( value = 'Tigre', inplace = True)

Volvemos a visualizar la cantidad de valores nulos para confirmar que no haya mas valores **Nan** en la variable 'place_name'

In [195]:
dfprop[dfprop.isnull().any(axis=1)].isna().sum()

operation                      0
property_type                  0
place_name                     0
place_with_parent_names        0
country_name                   0
                           ...  
expenses                   98647
properati_url                  0
description                    2
title                          0
image_thumbnail             2666
Length: 25, dtype: int64

## Variables 'surface_total_in_m2' y 'surface_covered_in_m2' - valores Nan
Analizamos los valores que tienen 'surface_covered_in_m2', pero no 'surface_total_in_m2'

In [196]:
dfprop.isnull().sum()

operation                      0
property_type                  0
place_name                     0
place_with_parent_names        0
country_name                   0
                           ...  
expenses                   98647
properati_url                  0
description                    2
title                          0
image_thumbnail             2666
Length: 25, dtype: int64

In [197]:
dfprop_surfaceanalysis = dfprop[(dfprop['surface_total_in_m2'].isna() == True) & (dfprop['surface_covered_in_m2'].isna() == False)]
dfprop_surfaceanalysis[['surface_total_in_m2', 'surface_covered_in_m2']].sample(10)

,surface_total_in_m2,surface_covered_in_m2
106849,NaN,30.0
47705,NaN,48.0
45239,NaN,76.0
66648,NaN,120.0
43305,NaN,120.0
72663,NaN,170.0
12013,NaN,20.0
89033,NaN,140.0
67423,NaN,54.0
32290,NaN,300.0


Reemplazamos los valores NaN de la columna 'surface_total_in_m2', asumiendo que en estos casos es igual al valor de 'surface_covered_in_m2'

In [198]:
dfprop.fillna({'surface_total_in_m2' : dfprop['surface_covered_in_m2']}, inplace = True)

Volvemos a constatar la cantidad de valores nulos en el dataframe:

In [199]:
dfprop.isnull().sum()

operation                      0
property_type                  0
place_name                     0
place_with_parent_names        0
country_name                   0
                           ...  
expenses                   98647
properati_url                  0
description                    2
title                          0
image_thumbnail             2666
Length: 25, dtype: int64

Repetimos la operación para los casos en los que no tenemos valor de 'surface_covered_in_m2', completando con el valor de 'surface_total_in_m2':

In [200]:
dfprop_surfaceanalysis = dfprop[(dfprop['surface_covered_in_m2'].isna() == True) & (dfprop['surface_total_in_m2'].isna() == False)]
dfprop_surfaceanalysis[['surface_total_in_m2', 'surface_covered_in_m2']].sample(10)

,surface_total_in_m2,surface_covered_in_m2
109603,36.0,NaN
53508,65.0,NaN
105977,95.0,NaN
13304,55.0,NaN
12131,40.0,NaN
29597,57.0,NaN
64318,109.0,NaN
39496,40.0,NaN
69813,55.0,NaN
20449,55.0,NaN


In [201]:
dfprop.fillna({'surface_covered_in_m2' : dfprop['surface_total_in_m2']}, inplace = True)


In [202]:
dfprop.isnull().sum()

operation                      0
property_type                  0
place_name                     0
place_with_parent_names        0
country_name                   0
                           ...  
expenses                   98647
properati_url                  0
description                    2
title                          0
image_thumbnail             2666
Length: 25, dtype: int64

Imputamos la variable 'price_per_m2' calculandola en base a 'price_aprox_us' y 'surface_total_in_m2' 

In [203]:

dfprop.loc[dfprop['price_usd_per_m2'].isna() &\
           (~dfprop['price_aprox_usd'].isna())&\
            (~dfprop['surface_total_in_m2'].isna())&\
            (dfprop['surface_total_in_m2']!=0.),\
           'price_usd_per_m2']= dfprop['price_aprox_usd']/dfprop['surface_total_in_m2']

In [204]:
dfprop.isnull().sum()

operation                      0
property_type                  0
place_name                     0
place_with_parent_names        0
country_name                   0
                           ...  
expenses                   98647
properati_url                  0
description                    2
title                          0
image_thumbnail             2666
Length: 25, dtype: int64

In [205]:
dfprop.shape

(112458, 25)

Eliminamos propiedades sin precio y con cantidad de metro=0

In [206]:

dfprop=dfprop.drop(dfprop.loc[(dfprop['price_usd_per_m2'].isna())&
                                 (dfprop['price'].isna())&
                                 (dfprop['surface_total_in_m2']==0)].index)

dfprop.shape

(112327, 25)

### Variable  'state_name' vs 'place_with_parent_names'
Notamos que la columna **'place_name'** contiene valores no que aportan una clara referencia de la ubicacion geografica del inmueble.

Por ejemplo
- place_name: 'Centro'
- place_with_parent_names: '|Argentina|Buenos Aires Costa Atlántica|Mar del Plata|Centro|'

Adicionalmente, los valores contenidos por esta columna no tienen el mismo valor referencial, lo cual difculta entender la precision de dicha columna. 

Por ello utilizamos **'place_with_parent_names'** que nos permite definir correctamente la ubicacion geografica del inmueble. 

Por lo tanto, separamos el texto y generaremos una nueva columna quedando el dataset con esta estructura.

- country_name: Pais
- state_name: provincia (salvo buenos aires que indicara la zona)
- district_name: localidad o partido
- city_name: barrio


In [207]:
# Split por el caracter "|"
dfplacename=[sublista.split('|') for sublista in dfprop['place_with_parent_names']]

# De cada una de las listas quitamos el ultimo y el primer caracter, ya que son vacios
# ['', 'Argentina', 'Bs.As. G.B.A. Zona Norte', 'Vicente López', 'Munro', '']
dfplacename=[sublista[1:5] for sublista in dfplacename]


# Se extrae el nombre de distrito o localidad en una nueva variable denominada 'district_name'
dfprop['district_name']=[str.strip(sublista[2]) if str.strip(sublista[2])!='' else np.nan for sublista in dfplacename]


# Se extrae el nombre de la ciudad o barrio en una nueva variable denominada 'city_name'
dfprop['city_name']=[str.strip(sublista[3]) if (len(sublista)>3  and sublista[3]) else np.nan for sublista in dfplacename]

# Finalmente columnas a ser utilizada seran:
# 'country_name'
# 'state_name'
# 'district_name'
#  'city_name'
# A partir de esas hacer las imputaciones

dfprop[['country_name','state_name','district_name','city_name']].sample(10)

,country_name,state_name,district_name,city_name
20087,Argentina,Bs.As. G.B.A. Zona Norte,Escobar,Belén de Escobar
83576,Argentina,Bs.As. G.B.A. Zona Sur,Ezeiza,Canning
103047,Argentina,Córdoba,Córdoba,NaN
37217,Argentina,Bs.As. G.B.A. Zona Norte,Vicente López,Olivos
47456,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Macrocentro
67586,Argentina,Bs.As. G.B.A. Zona Sur,Lomas de Zamora,Banfield
83754,Argentina,Bs.As. G.B.A. Zona Sur,Esteban Echeverría,NaN
27641,Argentina,Bs.As. G.B.A. Zona Norte,San Fernando,NaN
111264,Argentina,Córdoba,Córdoba,NaN
90437,Argentina,Bs.As. G.B.A. Zona Sur,Lanús,Valentín Alsina


### Eliminamos inmuebles sin nombre del distrito

Quitamos casos sin district_name ya que no nos permiten brindar un valor de referencia debido a que no tienen una zona claramente indicada   

In [208]:
mask_nodistrict_nocity_silatlon=(dfprop['district_name'].isna()==True)
dfprop=dfprop.loc[~mask_nodistrict_nocity_silatlon]
dfprop.shape

(108003, 27)

### Eliminamos las propiedades que por 'district_name' no lleguen a 30 operaciones

In [209]:
dfprop.groupby(by=['district_name']).count()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,city_name
district_name,,,,,,,,,,,,,,,,,,,,,
Abasto,103,103,103,103,103,103,0,68,68,68,...,92,88,17,61,26,103,103,103,97,0
Achiras,7,7,7,7,7,7,7,7,7,7,...,1,1,1,7,0,7,7,7,7,0
Adolfo Alsina,8,8,8,8,8,8,8,3,3,3,...,0,0,0,8,0,8,8,8,8,0
Agronomía,39,39,39,39,39,39,39,39,39,39,...,36,35,2,32,8,39,39,39,39,0
Agua de Oro,6,6,6,6,6,6,6,2,2,2,...,3,3,0,1,0,6,6,6,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Villarino,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,1,1,1,1,1
Vistalba,1,1,1,1,1,1,1,1,1,1,...,0,0,0,1,0,1,1,1,1,0
Wenceslao Escalante,1,1,1,1,1,1,1,1,1,1,...,1,0,0,0,0,1,1,1,1,0


In [210]:
#Creamos dataframe con un valor booleano que indica si district tiene menos de 30 propiedades (True) o no (False).
dfprop_districtless30locations=dfprop.groupby('district_name')[['operation']].count()<30

# Renombramos la columna  
dfprop_districtless30locations.rename(columns={'district_name':'district_name','operation':'less_30_locations'},inplace=True)

# Hacemos un merge con dfprop para marcar cada una de las propiedades
dfprop=pd.merge(dfprop,dfprop_districtless30locations,how='left',on='district_name')
    
# Generamos un nuevo Dataframe para almacenar las propiedades correspondientes a district con menos de 30 propiedades
dfprop_fewlocations=dfprop.loc[dfprop['less_30_locations']==True]

# Eliminamos del dataframe proncipal dichas ubiaciones
dfprop=dfprop.loc[dfprop['less_30_locations']==False]

#Eliminamos las columnas de ambos dataframes
dfprop_fewlocations.drop(['less_30_locations'],axis=1,inplace=True)
dfprop.drop(['less_30_locations'],axis=1,inplace=True)

#Verificamos la cantidad de ubicaciones dentro del dataframe auxiliar
dfprop_fewlocations.shape

(1895, 27)

In [211]:
dfprop.groupby(by=['district_name']).count()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,city_name
district_name,,,,,,,,,,,,,,,,,,,,,
Abasto,103,103,103,103,103,103,0,68,68,68,...,92,88,17,61,26,103,103,103,97,0
Agronomía,39,39,39,39,39,39,39,39,39,39,...,36,35,2,32,8,39,39,39,39,0
Almagro,1120,1120,1120,1120,1120,1120,1120,920,920,920,...,941,921,149,706,284,1120,1120,1120,1119,0
Almirante Brown,1520,1520,1520,1520,1520,1520,813,527,527,527,...,1070,999,7,156,93,1520,1520,1520,1454,1481
Avellaneda,628,628,628,628,628,628,615,396,396,396,...,581,547,28,292,62,628,628,628,627,404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Villa Real,36,36,36,36,36,36,36,35,35,35,...,36,36,0,17,2,36,36,36,36,0
Villa Santa Rita,55,55,55,55,55,55,55,54,54,54,...,54,54,11,39,2,55,55,55,55,0
Villa Urquiza,1607,1607,1607,1607,1607,1607,1607,1068,1068,1068,...,1333,1308,116,554,184,1607,1607,1607,1571,0


Corroboramos que el dataframe 'dfprop_fewlocations' se hayan agregado las propiedaes excluidas 

Corroboramos la cantidad de nan en la variable 'price_usd_per_m2'

In [212]:
dfprop.isna().sum()

operation                      0
property_type                  0
place_name                     0
place_with_parent_names        0
country_name                   0
                           ...  
description                    2
title                          0
image_thumbnail             2337
district_name                  0
city_name                  68162
Length: 27, dtype: int64

Habiendo eliminado 'district_names' con menos de 30 operaciones, procedemos a quitar aquellos registros intra-district, donde los NaN alcancen o superen el 50% de total de operaciones.

In [213]:
# Obtenemos un dataframe con las propiedades sin precio y con district_name 
dfsin_precio=dfprop.loc[dfprop['price_usd_per_m2'].isna() &\
                        (~dfprop['district_name'].isna())].groupby(['state_name','district_name'])[['operation']].count()

# Obtenemos un dataframe con las propiedades con precio y con district_name
dfcon_precio=dfprop.loc[(~dfprop['price_usd_per_m2'].isna()) &\
                        (~dfprop['district_name'].isna())].groupby(['state_name','district_name'])[['operation']].count()

# Unimos ambos dataframe y agrupamos para obtener la cantidad de propiedades sin y con precio para cada district_name
#Agrupamos por state_name y district_name para evitar que distritos de diferentes provincias se agrupen indebidamente 
df_merge=pd.merge(dfsin_precio,dfcon_precio,how='left',on=['state_name','district_name'])

# Renombro columnas a efectos practicos en la interpretacion
df_merge.rename(columns={'operation_x':'sin precio','operation_y':'con precio'},inplace=True)

# Creo una columna para obtener el total de propiedades
df_merge['total']=df_merge['con precio']+df_merge['sin precio']

# Calculo el % de propiedaes sin precio respecto al total de propiedades
df_merge['%']=df_merge['sin precio']/df_merge['total']

# Filtramos los distritos cuya relacion con la cantidad total de propiedades sea mayor a 0.5
df_merge=df_merge.loc[df_merge['%']>0.5]

#Hacemos un merge con dfprop para que las propiedades a ser removidas queden marcadas
dfpropdelete=pd.merge(dfprop,df_merge,how='left',on=['state_name','district_name'])

# Filtro el dataframe para obtener las propiedades a ingresar en dfprop 
dfprop= dfpropdelete.loc[dfpropdelete['total'].isna()|(~dfpropdelete['total'].isna())&(~dfpropdelete['price_usd_per_m2'].isna())]

# eliminon las columnas utilizadas en este analisis
dfprop.drop(['sin precio','con precio','%','total'],axis=1,inplace=True)

# Filtro el dataframe para obtener las propiedades excluidas 
dfpropdelete= dfpropdelete.loc[(~(dfpropdelete['total'].isna()|(~dfpropdelete['total'].isna())&(~dfpropdelete['price_usd_per_m2'].isna())))]

# elimino las columnas utilizadas en este analisis
dfpropdelete.drop(['sin precio','con precio','%','total'],axis=1,inplace=True)

# Agrego las propiedades excluidas al dataframe 
dfprop_fewlocations=dfprop_fewlocations.append(dfpropdelete)

dfprop.shape

(103889, 27)

In [214]:
dfprop.groupby(by=['district_name']).count()

,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,city_name
district_name,,,,,,,,,,,,,,,,,,,,,
Abasto,103,103,103,103,103,103,0,68,68,68,...,92,88,17,61,26,103,103,103,97,0
Agronomía,39,39,39,39,39,39,39,39,39,39,...,36,35,2,32,8,39,39,39,39,0
Almagro,1120,1120,1120,1120,1120,1120,1120,920,920,920,...,941,921,149,706,284,1120,1120,1120,1119,0
Almirante Brown,1520,1520,1520,1520,1520,1520,813,527,527,527,...,1070,999,7,156,93,1520,1520,1520,1454,1481
Avellaneda,628,628,628,628,628,628,615,396,396,396,...,581,547,28,292,62,628,628,628,627,404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Villa Real,36,36,36,36,36,36,36,35,35,35,...,36,36,0,17,2,36,36,36,36,0
Villa Santa Rita,55,55,55,55,55,55,55,54,54,54,...,54,54,11,39,2,55,55,55,55,0
Villa Urquiza,1607,1607,1607,1607,1607,1607,1607,1068,1068,1068,...,1333,1308,116,554,184,1607,1607,1607,1571,0


Corroboramos que el dataframe 'dfprop_fewlocations' se hayan agregado las propiedaes excluidas 

In [215]:
dfprop_fewlocations.shape

(4114, 27)

Corroboramos la cantidad de nan en la variable 'price_usd_per_m2'

In [216]:
dfprop.isna().sum()

operation                      0
property_type                  0
place_name                     0
place_with_parent_names        0
country_name                   0
                           ...  
description                    2
title                          0
image_thumbnail             2295
district_name                  0
city_name                  66678
Length: 27, dtype: int64

In [217]:
dfprop.groupby(by='state_name').count()

,operation,property_type,place_name,place_with_parent_names,country_name,geonames_id,lat-lon,lat,lon,price,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,district_name,city_name
state_name,,,,,,,,,,,,,,,,,,,,,
Bs.As. G.B.A. Zona Norte,24573,24573,24573,24573,24573,14602,11188,11188,11188,23326,...,21868,1772,6870,2980,24573,24572,24573,24275,24573,17211
Bs.As. G.B.A. Zona Oeste,7914,7914,7914,7914,7914,6910,4484,4484,4484,6967,...,6061,557,4043,608,7914,7914,7914,7637,7914,6321
Bs.As. G.B.A. Zona Sur,12591,12591,12591,12591,12591,8461,5214,5214,5214,11504,...,9513,235,2428,1725,12591,12591,12591,12201,12591,10532
Buenos Aires Costa Atlántica,9232,9232,9232,9232,9232,8228,7600,7600,7600,8600,...,6608,147,7350,416,9232,9232,9232,9159,9232,1128
Buenos Aires Interior,1534,1534,1534,1534,1534,1482,958,958,958,1410,...,1131,8,626,64,1534,1534,1534,1519,1534,650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Río Negro,254,254,254,254,254,254,152,152,152,251,...,178,15,129,6,254,254,254,221,254,213
Salta,91,91,91,91,91,91,74,74,74,69,...,55,9,37,0,91,91,91,90,91,0
San Luis,152,152,152,152,152,152,79,79,79,134,...,110,1,25,5,152,152,152,152,152,0


In [218]:
#Creamos dataframe con un valor booleano que indica si la provincia tiene menos de 30 propiedades (True) o no (False).
dfprop_stateless30locations=dfprop.groupby('state_name')[['operation']].count()<30

# Renombramos la columna  
dfprop_stateless30locations.rename(columns={'state_name':'state_name','operation':'less_30_locations'},inplace=True)

# Hacemos un merge con dfprop para marcar cada una de las propiedades
dfprop=pd.merge(dfprop,dfprop_stateless30locations,how='left',on='state_name')
    
# Generamos un nuevo Dataframe para almacenar las propiedades correspondientes a provincias con menos de 30 propiedades
dfprop_fewlocations=dfprop.loc[dfprop['less_30_locations']==True]

# Eliminamos del dataframe proncipal dichas ubiaciones
dfprop=dfprop.loc[dfprop['less_30_locations']==False]

#Eliminamos las columnas de ambos dataframes
dfprop_fewlocations.drop(['less_30_locations'],axis=1,inplace=True)
dfprop.drop(['less_30_locations'],axis=1,inplace=True)

#Verificamos la cantidad de ubicaciones dentro del dataframe auxiliar
dfprop_fewlocations.shape

(44, 27)

In [219]:
dfprop.groupby(by='state_name').count()

,operation,property_type,place_name,place_with_parent_names,country_name,geonames_id,lat-lon,lat,lon,price,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,district_name,city_name
state_name,,,,,,,,,,,,,,,,,,,,,
Bs.As. G.B.A. Zona Norte,24573,24573,24573,24573,24573,14602,11188,11188,11188,23326,...,21868,1772,6870,2980,24573,24572,24573,24275,24573,17211
Bs.As. G.B.A. Zona Oeste,7914,7914,7914,7914,7914,6910,4484,4484,4484,6967,...,6061,557,4043,608,7914,7914,7914,7637,7914,6321
Bs.As. G.B.A. Zona Sur,12591,12591,12591,12591,12591,8461,5214,5214,5214,11504,...,9513,235,2428,1725,12591,12591,12591,12201,12591,10532
Buenos Aires Costa Atlántica,9232,9232,9232,9232,9232,8228,7600,7600,7600,8600,...,6608,147,7350,416,9232,9232,9232,9159,9232,1128
Buenos Aires Interior,1534,1534,1534,1534,1534,1482,958,958,958,1410,...,1131,8,626,64,1534,1534,1534,1519,1534,650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Río Negro,254,254,254,254,254,254,152,152,152,251,...,178,15,129,6,254,254,254,221,254,213
Salta,91,91,91,91,91,91,74,74,74,69,...,55,9,37,0,91,91,91,90,91,0
San Luis,152,152,152,152,152,152,79,79,79,134,...,110,1,25,5,152,152,152,152,152,0


# Parte II: Análisis exploratorio 

Hasta el momento, visualizamos, limpiamos NaN, monedas, validamos ubicaciones spliteando place_with_parents_names y eliminamos registros que no conformaban una muestra robutas para crear estimadores.

En esta segunda etapa, damos el primer paso para tratar de completar los registros sin precio por mt2, pero antes de realizar este ajuste, procedemos a realizar un analisis exploratorio de la variable.

En la proxima seccion, abordaremos los valores outliers. Para ello, le asignaremos a cada registro un ratio que queda definido por el cociente entre el valor puntual del m2 y la media del district.

In [220]:
#Agrupamos por districto y ciudad, para calcular media y desvio. Dejamos fuera valores NaN.
dfprop_group = dfprop.groupby(by = ['state_name',"district_name","city_name"])["price_usd_per_m2"].describe()
dfprop_group_nona = dfprop_group.dropna()
dfprop_group_nona

count  \
state_name               district_name           city_name                        
Bs.As. G.B.A. Zona Norte Escobar                 Acacias Blancas           34.0   
                                                 Altos de Matheu            2.0   
                                                 Barrio Acacias             3.0   
                                                 Barrio Alamo Alto          2.0   
                                                 Barrio Araucarias          4.0   
...                                                                         ...   
Capital Federal          Palermo                 Palermo Chico            189.0   
                                                 Palermo Hollywood        453.0   
                                                 Palermo Soho             307.0   
                                                 Palermo Viejo             36.0   
Río Negro                San Carlos de Bariloche San Carlos de Bariloche  213.0   

                                                                                 mean  \
state_name               district_name           city_name                              
Bs.As. G.B.A. Zona Norte Escobar                 Acacias Blancas          1197.592259   
                                                 Altos de Matheu           910.000000   
                                                 Barrio Acacias           1769.553404   
                                                 Barrio Alamo Alto        6800.000000   
                                                 Barrio Araucarias        2065.750384   
...                                                                               ...   
Capital Federal          Palermo                 Palermo Chico            4827.114886   
                                                 Palermo Hollywood        3181.969888   
                                                 Palermo Soho             3241.420316   
                                                 Palermo Viejo            2998.865147   
Río Negro                San Carlos de Bariloche San Carlos de Bariloche  2441.779888   

                                                                                  std  \
state_name               district_name           city_name                              
Bs.As. G.B.A. Zona Norte Escobar                 Acacias Blancas           331.826156   
                                                 Altos de Matheu            14.142136   
                                                 Barrio Acacias            489.870225   
                                                 Barrio Alamo Alto        7919.595949   
                                                 Barrio Araucarias         715.478509   
...                                                                               ...   
Capital Federal          Palermo                 Palermo Chico            2421.761908   
                                                 Palermo Hollywood         900.170545   
                                                 Palermo Soho             1078.893477   
                                                 Palermo Viejo            1023.589642   
Río Negro                San Carlos de Bariloche San Carlos de Bariloche  2262.431303   

                                                                                  min  \
state_name               district_name           city_name                              
Bs.As. G.B.A. Zona Norte Escobar                 Acacias Blancas           467.741935   
                                                 Altos de Matheu           900.000000   
                                                 Barrio Acacias           1289.752650   
                                                 Barrio Alamo Alto        1200.000000   
                                                 Barrio Araucarias        1654.411765   
...                                                    

In [221]:
# Generamos una key de zona
dfprop['district_name_key']=dfprop['state_name']+"-"+\
                            dfprop['district_name']+"-"+\
                            dfprop['city_name'].fillna(value='')




# Agrupamos por key y sacamos precio promedio
dfprop_quartile=dfprop.groupby(by = ['district_name_key'])["price_usd_per_m2"].describe()
dfprop_quartile['district_name_key']=dfprop_quartile.index

In [222]:
dfprop_group_quartiles=dfprop_quartile[['25%','75%','district_name_key']]
dfprop_group_quartiles['cota_inf']=dfprop_group_quartiles['25%']-((dfprop_group_quartiles['75%']-dfprop_group_quartiles['25%'])*1.5)
dfprop_group_quartiles['cota_sup']=dfprop_group_quartiles['75%']+((dfprop_group_quartiles['75%']-dfprop_group_quartiles['25%'])*1.5)
dfprop_group_quartiles=dfprop_group_quartiles[['cota_inf','cota_sup']]
dfprop_group_quartiles

/opt/conda/envs/geopandas/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/envs/geopandas/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,cota_inf,cota_sup
district_name_key,,
Bs.As. G.B.A. Zona Norte-Escobar-,191.498481,2296.344931
Bs.As. G.B.A. Zona Norte-Escobar-Acacias Blancas,469.429348,1927.617754
Bs.As. G.B.A. Zona Norte-Escobar-Altos de Matheu,890.000000,930.000000
Bs.As. G.B.A. Zona Norte-Escobar-Barrio Acacias,785.510140,2743.819966
Bs.As. G.B.A. Zona Norte-Escobar-Barrio Alamo Alto,-4400.000000,18000.000000
...,...,...
Santa Fe-San Lorenzo-,-583.115312,2279.653336
Santa Fe-Santa Fe-,-877.076675,3182.185757
Santa Fe-Santo Tomé-,-1429.376772,4009.528463


In [223]:
dfprop=pd.merge(dfprop,dfprop_group_quartiles,how='left',on='district_name_key')

In [228]:
dfprop=dfprop.loc[(dfprop['price_usd_per_m2']>dfprop['cota_inf'])&(dfprop['price_usd_per_m2']<dfprop['cota_sup'])|(dfprop['price_usd_per_m2'].isna())]

In [233]:
dfprop.columns

Index(['operation', 'property_type', 'place_name', 'place_with_parent_names',
       'country_name', 'state_name', 'geonames_id', 'lat-lon', 'lat', 'lon',
       'price', 'currency', 'price_aprox_local_currency', 'price_aprox_usd',
       'surface_total_in_m2', 'surface_covered_in_m2', 'price_usd_per_m2',
       'price_per_m2', 'floor', 'rooms', 'expenses', 'properati_url',
       'description', 'title', 'image_thumbnail', 'district_name', 'city_name',
       'district_name_key', 'cota_inf', 'cota_sup'],
      dtype='object')

# Proceso de eliminacion de Outliers (en progreso)

paso uno, incluir quartil 1 y 3 (de price_usd_per_m2), en cada linea del DF original.
Luego, operar por linea, si el valor individual de la operacion se encuentra por debajo o por encima de las cotas. Las cotas quedan definidas:
Cota Minima: Q1 - 1.5*(Q3-Q1)
Cota Máxima: Q3 + 1.5*(Q3-Q1)
Luego se realiza un nueva visualizacion, observando la correccion en medias y desvios.
Con esto finalizado, se da paso al modulo de Buffer.

# Predictivos - Drivers

A fines de no ampliar el codigo, comentamos el trabajo realizado.
COn la libreria WordCloud identificamos palabras que se repetian e indentificamos "terminso positivos", que pudieran explicar un incrmento del costo promedio.
Luego creamos variables dummies (0=no contiene 1=contiene), para plotear en forma boxplot el valor del m2 segun contiene o no la variable dummy.
No se observó relacion contundente.

# Predictivos - Buffers 

Imputacion de Price_USD_M2 con valor NaN

1- Asignar price por ubicacion geografica (buffer-reducido)

2- Asignar price por ubicacion geografica (buffer-ampliado)

3- Crear un cluster por country-state-district-city e imputar el precio por m2 promedio a los registros que no tengan Nan en city_name

4- Crear un cluster por country-state-district e imputar el precio por m2 promedio a los registros que tengan Nan en city_name y no tengan NaN en district_name

5- Eliminar registros que no se les haya podido imputar precio


In [236]:
# Cargo propiedades en el mapa usando lat lon
import geopandas
import shapely.wkt
import rtree
pd.options.display.max_rows = 10
import matplotlib.pyplot as plt
%matplotlib inline

from shapely.geometry import Point

# Funcion para crear puntos a partir del lat-lon
def from_x_y(df, x, y):
    gdf = geopandas.GeoDataFrame(df.drop([x, y], axis=1),
                                crs={'init': 'epsg:4326'},
                                geometry=[Point(xy) for xy in zip(df[x], df[y])])
    return gdf


def getProyectionBuffer(locations,buff):
    loc_gkba = locations.to_crs(crs = "+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs")
    loc_gkba_buff=loc_gkba.copy()
    loc_gkba_buff.geometry = loc_gkba.buffer(buff)
    return loc_gkba_buff
    
# Funcion para obtener el precio promedio por buffer
def getMeanBuffer(dfprop_noprice_loc,location_price_gkba_buff,indexprop,variableToEstimate,buffer):
    
    locations_noprice=from_x_y(dfprop_noprice_loc.loc[[indexprop]], "lon", "lat")
    
     # Creo un buffer 
    locations_noprice_gkba_buff=getProyectionBuffer(locations_noprice,buffer)    
    
    # Borramos la columna 'price_aprox_usd' del dataframe que no tiene precio
    locations_noprice_gkba_buff.drop(variableToEstimate,axis=1,inplace=True)

    try:
        #Busco la interseccion
        intersection = geopandas.overlay(locations_noprice_gkba_buff,location_price_gkba_buff, how = "intersection")

        price_mean=intersection[variableToEstimate].mean()

        return float(price_mean)
    except:
        #Si no encuentro la interseccion, devuelvo 0
        return float(0)

    
def estimateVariable(dfToEstimate,variableToEstimate):

    # Obtengo dataframe de todas las propiedades de una localidad SIN precio
    dfprop_noprice=dfToEstimate.loc[((~dfToEstimate['lat'].isna()) &\
                               (~dfToEstimate['lon'].isna()) &\
                               dfToEstimate[variableToEstimate].isna() &\
                               (~dfToEstimate['district_name'].isna()))]
    print(dfprop_noprice.shape)
    # Obtengo dataframe de todas las propiedades de una localidad CON precio
    dfprop_price=dfToEstimate.loc[((~dfToEstimate['lat'].isna()) &\
                             (~dfToEstimate['lon'].isna()) &\
                             (~dfToEstimate[variableToEstimate].isna())&\
                             (~dfToEstimate['district_name'].isna()))]


    # Creo key de listado de localidades
    dfprop_noprice['district_name_key']=dfprop_noprice['country_name']+"-"+\
                                      dfprop_noprice['state_name']+"-"+\
                                      dfprop_noprice['district_name']


    dfprop_price['district_name_key']=dfprop_price['country_name']+"-"+\
                                      dfprop_price['state_name']+"-"+\
                                      dfprop_price['district_name']

    crs = {'init': 'epsg:4326'} 

    # Filtro por una localidad
    # Repito la operacion por localidad 
    for proploc in dfprop_noprice['district_name_key'].unique():
        i=1

        # Extraigo en 2 dataframes las propiedades sin precio y con precio
        dfprop_noprice_loc= dfprop_noprice.loc[dfprop_noprice['district_name_key']==proploc]
        dfprop_price_loc= dfprop_price.loc[dfprop_price['district_name_key']==proploc]

        # Cargo todas las propiedades con precio en geopandas
        locations_with_price = from_x_y(dfprop_price_loc.loc[:,[variableToEstimate,'lon','lat']], "lon", "lat")

        # Creo la proyeccion de las propiedades con precio 
        locations_with_price_gkba_buff=getProyectionBuffer(locations_with_price,1) 
        
        print('Localidad: '+proploc + 'Cantidad de propiedades:'+ str(dfprop_noprice_loc['operation'].count()))
        # Busco el precio para cada propiedad de esa localidad 
        for iprop in dfprop_noprice_loc[:].index:
            
            #print('\tBusco precio por 1Km')
            # Busco y asigno el precio promedio de las propiedades a 1km de distancia 
            mean_price=getMeanBuffer(dfprop_noprice_loc,locations_with_price_gkba_buff,iprop,variableToEstimate,1000)

            # Si no encuentro un precio promedio de las propiedades a 1.5km de distancia 
            if(mean_price==0):
                #print('\tBusco precio por 1.5Km')
                mean_price=getMeanBuffer(dfprop_noprice_loc,locations_with_price_gkba_buff,iprop,variableToEstimate,1500)


            city_name=str(dfprop_noprice_loc.loc[:,'city_name'].loc[iprop])
            # Si no encuentro un precio promedio y la propiedad tiene barrio, busco el promedio para el barrio  
            if((mean_price==0) & (city_name!='nan')):
                #print('\tCalculo por barrio',city_name)
                # Creo mascara para solo calcular la media de las propiedades con precio en el mismo barrio
                city_name_mask=(dfprop_price_loc.loc[:,'city_name']==city_name)
                #VArifico que haya alguna propiedad con precio para ese barrio
                if(dfprop_price_loc[city_name_mask].empty==False):
                    # Aplico la mascara y calculo la media
                    #print('calculo media por barrio')
                    mean_price=dfprop_price_loc.loc[city_name_mask,variableToEstimate].median()
                else:
                    mean_price=0

             # Si no encuentro un precio promedio busco el promedio para la localidad  
            if(mean_price==0):
                #Calculo el precio promedio en base a propiedades en la misma localidad
               # print('\tCalculo por localidad')
                mean_price=dfprop_price_loc[variableToEstimate].median()

            print('\t'+str(i)+' - Mean Price= '+str(mean_price))

            dfToEstimate.loc[dfToEstimate.loc[[iprop]].index,variableToEstimate]=mean_price
            i=i+1
            
        print('---------------------------------------------') 
        
    return dfToEstimate
       


dfprop=estimateVariable(dfprop,'price_usd_per_m2')



(9157, 30)


/opt/conda/envs/geopandas/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/geopandas/lib/python3.7/site-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Localidad: Argentina-Bs.As. G.B.A. Zona Sur-La PlataCantidad de propiedades:226
	1 - Mean Price= 1781.3162536935893
	2 - Mean Price= 1685.662840577837
	3 - Mean Price= 1700.912208611637
	4 - Mean Price= 1258.2893645977406
	5 - Mean Price= 1690.999778703143
	6 - Mean Price= 1402.0915
	7 - Mean Price= 1618.8823062213792
	8 - Mean Price= 1006.1814615750421
	9 - Mean Price= 1669.385720743812
	10 - Mean Price= 1375.1217926181525
	11 - Mean Price= 1308.1512676972527
	12 - Mean Price= 911.5093604684199
	13 - Mean Price= 1440.5461893823435
	14 - Mean Price= 1449.3383341189822
	15 - Mean Price= 1449.3383341189822
	16 - Mean Price= 1449.3383341189822
	17 - Mean Price= 1612.9123749151886
	18 - Mean Price= 929.8930700849577
	19 - Mean Price= 1622.5313081674976
	20 - Mean Price= 1383.1572036086527
	21 - Mean Price= 1698.7016416403917
	22 - Mean Price= 1345.3201970443351
	23 - Mean Price= 1401.6256006431552
	24 - Mean Price= 1387.8114385381764
	25 - Mean Price= 1610.3958554973688
	26 - Mean Price= 1

	222 - Mean Price= 1542.978886390913
	223 - Mean Price= 1240.8454861302382
	224 - Mean Price= 992.2348973103034
	225 - Mean Price= 1668.3747278214244
	226 - Mean Price= 1591.5235303372885
---------------------------------------------
Localidad: Argentina-Capital Federal-LiniersCantidad de propiedades:91
	1 - Mean Price= 1763.7472537420015
	2 - Mean Price= 1761.7237785008044
	3 - Mean Price= 1761.7237785008044
	4 - Mean Price= 1761.7237785008044
	5 - Mean Price= 1665.0832215837302
	6 - Mean Price= 1705.0934188287358
	7 - Mean Price= 1739.152497087665
	8 - Mean Price= 1760.1162956922676
	9 - Mean Price= 1765.497290529297
	10 - Mean Price= 1742.2034735983777
	11 - Mean Price= 1770.3754725833803
	12 - Mean Price= 1819.137336838442
	13 - Mean Price= 1616.327595012055
	14 - Mean Price= 1775.9709885947605
	15 - Mean Price= 1768.053190858735
	16 - Mean Price= 1717.3179855393682
	17 - Mean Price= 1659.9744292804317
	18 - Mean Price= 1739.752221094092
	19 - Mean Price= 1739.752221094092
	20 - Me

	122 - Mean Price= 1000.4926048072738
	123 - Mean Price= 1728.4561945655628
	124 - Mean Price= 1049.419466190543
	125 - Mean Price= 1870.8782834701185
	126 - Mean Price= 1869.9027688861754
	127 - Mean Price= 1699.567213218048
	128 - Mean Price= 1823.6330613197304
	129 - Mean Price= 1694.6405277908068
	130 - Mean Price= 1335.0473227505759
	131 - Mean Price= 897.2004682854134
	132 - Mean Price= 1778.9266905400664
	133 - Mean Price= 1707.0186881644624
	134 - Mean Price= 1494.1132308423269
	135 - Mean Price= 1039.5310854978354
	136 - Mean Price= 1822.334475919057
	137 - Mean Price= 1822.334475919057
	138 - Mean Price= 920.7874429140245
	139 - Mean Price= 970.7182301375884
	140 - Mean Price= 1666.4071145701002
	141 - Mean Price= 1395.1986783439888
	142 - Mean Price= 1822.334475919057
	143 - Mean Price= 1770.321163461764
	144 - Mean Price= 1675.501931084312
	145 - Mean Price= 1673.6382860359447
	146 - Mean Price= 1787.1587489966264
	147 - Mean Price= 1729.0332946839394
	148 - Mean Price= 147

...

Exception ignored in: <function Handle.__del__ at 0x7fd43d258b70>
Traceback (most recent call last):
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
    self.destroy()
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 863, in destroy
    if self._ptr is not None:
AttributeError: 'IndexStreamHandle' object has no attribute '_ptr'
Exception ignored in: <function Handle.__del__ at 0x7fd43d258b70>
Traceback (most recent call last):
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
    self.destroy()
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 863, in destroy
    if self._ptr is not None:
AttributeError: 'IndexStreamHandle' object has no attribute '_ptr'


	1 - Mean Price= nan
---------------------------------------------
Localidad: Argentina-Capital Federal-Villa Santa RitaCantidad de propiedades:1
	1 - Mean Price= 2015.0668078174274
---------------------------------------------
Localidad: Argentina-Buenos Aires Costa Atlántica-CarilóCantidad de propiedades:13
	1 - Mean Price= 1235.6963086677597
	2 - Mean Price= 1654.7708006786352
	3 - Mean Price= 1879.6574904524136
	4 - Mean Price= 1486.0049207661343
	5 - Mean Price= 1522.5618832419618
	6 - Mean Price= 1537.6996159363678
	7 - Mean Price= 2211.014444555893
	8 - Mean Price= 587.42198654384
	9 - Mean Price= 1093.4337044534414
	10 - Mean Price= 1629.812382739212
	11 - Mean Price= 1808.063930608729
	12 - Mean Price= 1522.5618832419618
	13 - Mean Price= 2054.530542524131
---------------------------------------------
Localidad: Argentina-Mendoza-RivadaviaCantidad de propiedades:1
	1 - Mean Price= nan
---------------------------------------------


Exception ignored in: <function Handle.__del__ at 0x7fd43d258b70>
Traceback (most recent call last):
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
    self.destroy()
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 863, in destroy
    if self._ptr is not None:
AttributeError: 'IndexStreamHandle' object has no attribute '_ptr'
Exception ignored in: <function Handle.__del__ at 0x7fd43d258b70>
Traceback (most recent call last):
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
    self.destroy()
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 863, in destroy
    if self._ptr is not None:
AttributeError: 'IndexStreamHandle' object has no attribute '_ptr'


Localidad: Argentina-Córdoba-Santa MaríaCantidad de propiedades:13
	1 - Mean Price= nan
	2 - Mean Price= nan
	3 - Mean Price= nan
	4 - Mean Price= nan
	5 - Mean Price= nan
	6 - Mean Price= nan
	7 - Mean Price= nan
	8 - Mean Price= nan
	9 - Mean Price= nan
	10 - Mean Price= nan
	11 - Mean Price= nan
	12 - Mean Price= nan
	13 - Mean Price= nan
---------------------------------------------
Localidad: Argentina-Capital Federal-Villa OrtuzarCantidad de propiedades:1
	1 - Mean Price= 2343.0769361601674
---------------------------------------------
Localidad: Argentina-Santa Fe-Capitán BermúdezCantidad de propiedades:1
	1 - Mean Price= 520.1969627126406
---------------------------------------------
Localidad: Argentina-Córdoba-MiramarCantidad de propiedades:2
	1 - Mean Price= nan
	2 - Mean Price= nan
---------------------------------------------
Localidad: Argentina-Córdoba-Malvinas ArgentinasCantidad de propiedades:1
	1 - Mean Price= 1000.0
---------------------------------------------


Exception ignored in: <function Handle.__del__ at 0x7fd43d258b70>
Traceback (most recent call last):
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
    self.destroy()
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 863, in destroy
    if self._ptr is not None:
AttributeError: 'IndexStreamHandle' object has no attribute '_ptr'
Exception ignored in: <function Handle.__del__ at 0x7fd43d258b70>
Traceback (most recent call last):
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 875, in __del__
    self.destroy()
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtree/index.py", line 863, in destroy
    if self._ptr is not None:
AttributeError: 'IndexStreamHandle' object has no attribute '_ptr'
Exception ignored in: <function Handle.__del__ at 0x7fd43d258b70>
Traceback (most recent call last):
  File "/opt/conda/envs/geopandas/lib/python3.7/site-packages/rtr

Imputamos el valor de 'price_usd_per_m2' promedio del distrito en los casos cuyo valor es 'Nan' y no tienen lat-lon

In [238]:
# Generamos una key de zona
dfprop['district_name_key']=dfprop['country_name']+"-"+\
                            dfprop['state_name']+"-"+\
                            dfprop['district_name']

# Creamos un dataframe auxiliar
dfprop_price=dfprop


# Agrupamos por key y sacamos precio promedio
dfprop_price=dfprop_price.groupby('district_name_key')[['price_usd_per_m2']].mean()


# Renombro columnas a efectos practicos en la interpretacion
dfprop_price.rename(columns={'price_usd_per_m2':'price_usd_per_m2_district'},inplace=True)


#Unimos las tablas por left para agregar la columna precio por distrito
dfprop=pd.merge(dfprop,dfprop_price,how='left',on='district_name_key')

#Imputamos el precio promedio por district a los casos que tengan Nan y que tengan District
dfprop.loc[(dfprop['price_usd_per_m2'].isna() &\
          (~dfprop['district_name'].isna())),'price_usd_per_m2']=dfprop['price_usd_per_m2_district']


# Elimino las columnas utilizadas en este analisis
dfprop.drop(['price_usd_per_m2_district','district_name_key'],axis=1,inplace=True)

dfprop.shape

(99585, 29)

Eliminamos las propiedades a las cuales no se les pudo imputar un valor en 'price_usd_per_m2'


In [240]:
dfprop=dfprop.loc[(~dfprop['price_usd_per_m2'].isna())]

Imputamos el precio en base a 'price_usd_per_m2' y 'surface_total_in_m2'

In [241]:
#Asignamos la moneda USD 
dfprop.loc[dfprop['price'].isna() &\
           (~dfprop['price_usd_per_m2'].isna())&\
            (~dfprop['surface_total_in_m2'].isna()),\
           'currency']= 'USD'

#Calculamos el precio en dolares para las propiedades que tenemos la superficie total y el valor en USD por m2 
dfprop.loc[dfprop['price'].isna() &\
           (~dfprop['price_usd_per_m2'].isna())&\
            (~dfprop['surface_total_in_m2'].isna()),\
           'price']= dfprop['price_usd_per_m2']*dfprop['surface_total_in_m2']

dfprop.loc[dfprop['price_aprox_usd'].isna() &\
           (~dfprop['price_usd_per_m2'].isna())&\
            (~dfprop['surface_total_in_m2'].isna()),\
           'price_aprox_usd']= dfprop['price_usd_per_m2']*dfprop['surface_total_in_m2']

In [242]:
# Cantidad de registros sin precio m2 cuadrado ni precio 
dfpropa=dfprop.loc[:,\
            ['price','price_aprox_usd','currency','surface_total_in_m2','surface_covered_in_m2','price_per_m2','price_usd_per_m2']]

dfpropa[dfpropa.isnull().any(axis=1)].isna().sum()

price                        0
price_aprox_usd              0
currency                     0
surface_total_in_m2       6299
surface_covered_in_m2     6299
price_per_m2             22498
price_usd_per_m2             0
dtype: int64

In [243]:
dfprop.shape


(99580, 29)

In [245]:
dfprop.to_csv('properati_buffered.csv')